# 9.6 Homework: SQL and Databases
- Name: Congxin (David) Xu
- Computing ID: cx2rx

### Question 1
Write the following queries in SQL, using the university schema.

1. Find the ID and name of each student who has taken at least one Comp. Sci. course; make sure there are no duplicate names in the result.
2. Find the ID and name of each student who has not taken any course offered before 2017.
3. For each department, find the maximum salary of instructors in that department. You may assume that every department has at least one instructor.
4. Find the lowest, across all departments, of the per-department maximum salary computed by the preceding query.

<img src="University Schema.png" width="1000">

**Answer**

**Part 1**: Find the ID and name of each student who has taken at least one Comp. Sci. course; make sure there are no duplicate names in the result.
```{sql}
select distinct ID, name from student 
inner join (select ID, course_id, sec_id, semester, year from takes
            inner join (select course_id, sec_id, semester, year from section
                        left join (select course_id, dept_name from course
                                   where dept_name = 'Comp. Sci.') c
                        on section.course_id = c.course_id) s2
            on takes.course_id = s2.course_id
            and takes.sec_id = s2.sec_id
            and takes.semester = s2.semester
            and takes.year = s2.year) t2
on student.ID = t2.ID
```

**Part 2**: Find the ID and name of each student who has not taken any course offered before 2017.
```{sql}
select distinct ID, name from student 
inner join (select ID, course_id, sec_id, semester, year from takes
            inner join (select course_id, sec_id, semester, year from section
                        where year >= 2017) s2                        
            on takes.course_id = s2.course_id
            and takes.sec_id = s2.sec_id
            and takes.semester = s2.semester
            and takes.year = s2.year) t2
on student.ID = t2.ID
```

**Part 3**: For each department, find the maximum salary of instructors in that department. You may assume that every department has at least one instructor.
```{sql}
select name, dept_name from instructor
inner join (select dept_name, max(salary) as max_salary from instructor
            group by dept_name) m
on instructor.dept_name = m.dept_name
and instructor.salary = m.max_salary
```

**Part 4**: Find the lowest, across all departments, of the per-department maximum salary computed by the preceding query.
```{sql}
select top 1 name, dept_name, salary from instructor
inner join (select dept_name, max(salary) as max_salary from instructor
            group by dept_name) m
on instructor.dept_name = m.dept_name
and instructor.salary = m.max_salary
order by salary asc
```